In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.datasets import load_iris
from scipy.stats import spearmanr
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time

READING NSL-KDD DATASET AND CHECKING FOR CLASS IMBALANCE

In [2]:
df = pd.read_csv('KDDTrain+.csv')
# replace normal with 0 and anomaly with 1
df['class'] = df['class'].replace(['normal'], 0)
df['class'] = df['class'].replace(['anomaly'], 1)

CLEAINING AND PRE-PROCESSING OF THE DATASET

- NORMALIZING FUNCTION CREATION 

In [3]:
def normalize (df, exclude=[]):
    for col in df.columns:
        if col not in exclude:
            # use min-max normalization
            df[col] = (df[col] - df[col].min()) / (df[col].std())
    return df

- EXCLUDING FEATURES THAT CANNOT BE NORMALIZED AND NORMALIZING OTHERS

In [4]:
exclude = ['class', 'protocol_type', 'service', 'flag', 'land', 'logged_in', 'is_host_login', 'is_guest_login','type']
df = normalize(df, exclude)

- ASSIGNING VALUES TO STRING TYPE FEATURES
    - PROTOCOL_TYPE
    - SERVICE
    - FLAG
- DROPPING NAN TYPE FEATURE(NUM_OUTBOUND_CMD)

In [5]:
# assign 0 to tcp, 1 to udp, 2 to icmp
df['protocol_type'] = df['protocol_type'].replace(['tcp'], 0)
df['protocol_type'] = df['protocol_type'].replace(['udp'], 1)
df['protocol_type'] = df['protocol_type'].replace(['icmp'], 2)

# assign values to service
service = df['service'].unique()
service = np.sort(service)
service = service.tolist()
for i in range(len(service)):
    df['service'] = df['service'].replace([service[i]], i)
# assign values to flag
flag = df['flag'].unique()
flag = np.sort(flag)
flag = flag.tolist()
for i in range(len(flag)):
    df['flag'] = df['flag'].replace([flag[i]], i)
exclude.remove('type')
for i in exclude:
    df[i] = df[i].astype('int64')

cols = df.columns.tolist()

# remove num_outbound_cmds from df as it has NaN values
df = df.drop('num_outbound_cmds', axis=1)
ef = df.copy()

In [6]:
ef.drop('type', axis=1, inplace=True)

USING CHI-SQUARED STATISTIC TO GET THE TOP 20 FEATURES(TO COMPARE IT WITH SCC), PRINTING THE WORST 6 FEATURES TO BE REMOVED

In [7]:
cols = ef.columns.tolist()
selector = SelectKBest(score_func=chi2, k=20)
X_new = selector.fit_transform(ef, ef['class'])
selected_features_indices = np.where(selector.get_support())[0]
sel_col = []
for i in range(41):
    if i not in selected_features_indices:
        sel_col.append(cols[i])
print(sel_col)

['duration', 'protocol_type', 'src_bytes', 'dst_bytes', 'land', 'urgent', 'hot', 'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'is_host_login', 'is_guest_login', 'srv_count', 'srv_diff_host_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate']


REMOVING THE FEATURES

In [31]:
ef.drop(['protocol_type', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'is_host_login', 'is_guest_login', 'srv_count', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate','duration'], axis=1, inplace=True)

In [38]:
# save processed data to csv
ef.to_csv('kdd_processed_bin.csv', index=False)
